# K Nearest Neighbor in TensorFlow 2.0

I was asked this question and I really struggled with it so I decided to Google around and see what was out there. I found a few v1 solutions but nothing that used v2. So, here I am.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/ehennis/Blog/blob/master/TensorFlow/K_Nearest_Neighbor.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/ehennis/Blog/blob/master/TensorFlow/K_Nearest_Neighbor.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In [0]:
!pip install tensorflow==2.0.0-alpha0

In [2]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import numpy as np

from sklearn import datasets

print(tf.__version__)

2.0.0-alpha0


In [4]:
column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']
feature_names = column_names[:-1]
label_name = column_names[-1]

class_names = ['Iris setosa', 'Iris versicolor', 'Iris virginica']
print("Features: {}".format(feature_names))
print("Label: {}".format(label_name))

Features: ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
Label: species


**K Nearest Neighbor**  
I need to grab some *test* data in order to compare versus the *train* data.

In [5]:
iris_dataset = datasets.load_iris()

#Create X/Y arrays
x_vals = np.array([x[0:4] for x in iris_dataset.data])
y_vals = np.array(iris_dataset.target)
y_vals = np.eye(len(set(y_vals)))[y_vals]
x_vals = (x_vals - x_vals.min(0)) / x_vals.ptp(0)

np.random.seed(50)
#Grab 80%
train_indices = np.random.choice(len(x_vals), round(len(x_vals) * 0.8), replace=False)
#Test on the remaining 20%
test_indices = np.array(list(set(range(len(x_vals))) - set(train_indices)))

x_vals_train = x_vals[train_indices] #120 records
x_vals_test = x_vals[test_indices] #30 records
y_vals_train = y_vals[train_indices]
y_vals_test = y_vals[test_indices]

feature_number = len(x_vals_train[0]) #4 features

#Grab the 3 nearest neighbors
k=3 

#Grab a value to use to find the knn
orig_value = x_vals_test[0]

#Calculate the squared difference for each training record against
#the orig_valu
t_diffs = tf.math.squared_difference(x_vals_train, orig_value)
#Sum across the array (4 features) into a single value
t_diff_sums = tf.reduce_sum(t_diffs, axis=1)
#Negate the values since I really want the LOWEST values
elements, indices = tf.math.top_k(tf.negative(t_diff_sums),k=k)

#print(elements)
print(indices)
print('Original Element: %s' % orig_value)
print('1st Best Element: %s' % x_vals_train[indices[0]])
print('2nd Best Element: %s' % x_vals_train[indices[1]])
print('3rd Best Element: %s' % x_vals_train[indices[2]])


tf.Tensor([75 80 35], shape=(3,), dtype=int32)
Original Element: [0.11111111 0.5        0.05084746 0.04166667]
1st Best Element: [0.08333333 0.5        0.06779661 0.04166667]
2nd Best Element: [0.11111111 0.5        0.10169492 0.04166667]
3rd Best Element: [0.08333333 0.45833333 0.08474576 0.04166667]
